In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI
from Exercise05AI import Exercise05AI
from Exercise04AI import Exercise04AI

# Aufgabe 06: Minimax mit Alpha-Beta-Pruning und Memoisierung

Dieses Notebook implementiert den Minimax-Algorithmus mit Alpha-Beta-Pruning und Memoisierung.

In [ ]:
import chess_custom as chess
from typing import Any

class Exercise06AI(Exercise05AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning and memoization"""
    def __init__(self, player_name: str, search_depth: int = 3) -> None:
        super().__init__(player_name, search_depth)
        self.cache: dict[tuple, Any] = {}
        self.cache_hits = 0
    
    def reset(self) -> None:
        """Resets all internal variables"""
        super().reset()
        self.cache = {}
        self.cache_hits = 0

In [ ]:
from typing import Any, Callable

class Exercise06AI(Exercise06AI): # type: ignore
    
    @staticmethod
    def memoize_minimax(minimax: Callable):
        def minimax_memoized(self, board: chess.Board, depth: int, last_eval: int,
                             alpha: int = -Exercise06AI.LIMIT, beta: int = Exercise06AI.LIMIT):
            key = Exercise04AI.get_key(board, depth)
            if key in self.cache:
                self.cache_hits += 1
                return self.get_from_cache(minimax, key, board, depth, last_eval, alpha, beta)
            result = minimax(self, board, depth, last_eval, alpha, beta)
            self.store_in_cache(key, result, alpha, beta)
            return result

        return minimax_memoized
    
    @memoize_minimax
    def minimax(self, *args) -> int:
        """Memoized version of the Exercise05AI minimax implementation"""
        return super().minimax(*args)
    
    def store_in_cache(self, key: tuple, result: tuple, alpha: int, beta: int) -> None:
        """Stores the result of a minimax computation in the cache."""
        evaluation, move = result
        if evaluation <= alpha:
            self.cache[key] = ("≤", evaluation, move)
        elif evaluation < beta:
            self.cache[key] = ("=", evaluation, move)
        else:
            self.cache[key] = ("≥", evaluation, move)
            
    def get_from_cache(self, minimax: Callable, key: tuple, board: chess.Board,
                       depth: int, last_eval: int, alpha: int, beta: int) -> tuple:
        """Gets a result from the cache if possible."""
        flag, evaluation, move = self.cache[key]
        if flag == "=":
            return evaluation, move
        elif flag == "≤":
            if evaluation <= alpha:
                return evaluation, move
            elif evaluation < beta:
                result = minimax(self, board, depth, last_eval, alpha, evaluation)
                self.store_in_cache(key, result, alpha, evaluation)
                return result
            else:
                result = minimax(self, board, depth, last_eval, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
        else:
            if evaluation <= alpha:
                result = minimax(self, board, depth, last_eval, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
            elif evaluation < beta:
                result = minimax(self, board, depth, last_eval, evaluation, beta)
                self.store_in_cache(key, result, evaluation, beta)
                return result
            else:
                return evaluation, move

## Debugging Bereich

Die folgenden Zellen enthalten Code zum Testen der oben implementierten Funktionen.

In [ ]:
from Exercise03AI import Exercise03AI

In [ ]:
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 w - - 0 36")
board.push(chess.Move.from_uci("h2h1"))
board

In [ ]:
%%time
DEPTH = 4
player3 = Exercise03AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player3.get_next_middle_game_move(board)
print(move)

In [ ]:
%%time
DEPTH = 4
player5 = Exercise05AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player5.get_next_middle_game_move(board)
print(move)

In [ ]:
%%time
DEPTH = 4
player6 = Exercise06AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player6.get_next_middle_game_move(board)
print(move)
print(player6.cache_hits)

In [ ]:
from IPython.display import clear_output, display
player5 = Exercise05AI("Testplayer", 3)
player3 = Exercise03AI("Testplayer", 3)
move = player3.random.choice(list(board.legal_moves))
board.push(move)
for _ in range(10):
    move_1, eval_1 = player3.get_next_middle_game_move(board)
    player3.last_evaluation = None
    print(move_1, eval_1, player3.last_evaluation)
    move_2, eval_2 = player5.get_next_middle_game_move(board)
    player5.last_evaluation = None
    print(move_2, eval_2, player5.last_evaluation)
    move = player3.random.choice(list(board.legal_moves))
    board.push(move)

In [ ]:
%%time
DEPTH = 3
board = chess.Board()
#board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
for _ in range(10):
    board.push(sorted(board.legal_moves, key=lambda move: move.uci())[0])
    move = player.get_next_middle_game_move(board)
    print(move)

In [ ]:
from os.path import join

with open(join("..", "games", "2022-01-15_20-18-57-907777.pgn")) as pgn:
    first_game = chess.pgn.read_game(pgn)

# Iterate through all moves and play them on a board.
board = first_game.board()
for move in first_game.mainline_moves():
    board.push(move)
for i in range(220):
    board.pop()

In [ ]:
%%time
DEPTH = 2
#board = chess.Board()
#board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.get_next_middle_game_move(board)
print(move)
player.cache_hits

In [ ]:
board = chess.Board("7B/pbpk4/1p6/2n1pp2/6p1/4P3/P1P2PNP/RNq1K1R1 w - - 0 1")
player = Exercise04AI("Testplayer", 3)
print(player.evaluate(board))
board